In [ ]:
!pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cpu

In [ ]:
!pip3 install torch torchvision

In [ ]:
import os
import glob

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.optim import SGD, Adam

from torchvision.io import read_image
from torchvision.transforms import v2

from tqdm.notebook import tqdm
from PIL import Image

## 1. Daten Encodiern und Laden

In PyTorch mit einem Dataset + Dataloaders. Zusätzlich brauchen wir noch Funktionen um die Klassen der Quallen zu encodieren + decodieren.

1. Encoding / decoding Funktionen implementieren
2. Database Klasse erstellen (erstmal nur train+val) ALTERNATIV ImageFolder von torchvision.datasets nutzen, falls ja wie Abwägung begründen
3. Was machen die Vordefinierten transformationen?
4. Dataloaders definieren.
    - WICHTIG, shuffle flag nicht vergessen. Warum?
    - train batchsize setzen. Warum? Welche tradeoffs bei hoher / niedriger batchsize
    - Brauchen wir batchsize & shuffle flag bei train und val?

### Hinweise
- Was genau braucht die loss function später? Wie kommen wir von dem String der Quallen Klassen zu dem was die loss function braucht?
- Dataset Doku: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files
    - ImageFolder Doku: https://pytorch.org/vision/0.16/generated/torchvision.datasets.ImageFolder.html?highlight=imagefolder#torchvision.datasets.ImageFolder
- Ein Dataset braucht eine Liste mit informationen (Image location + Klasse)
- `read_image` von `torchvision.io` erstellt lädt ein Bild als Tensor

In [ ]:
"""
Helper code
"""

data_dir_train = 'data/jellyfish/Train_Test_Valid/Train'
data_dir_val = 'data/jellyfish/Train_Test_Valid/valid'

labels_train = []
labels_val = []

def get_last_folder(path):
    return os.path.basename(os.path.normpath(path))

def get_labels(root_dir):
    res = []
    for s in glob.glob(root_dir + '/*'):
        cls = get_last_folder(s)
        for f in glob.glob(s + '/*'):
            res.append((cls, f))
    return res

labels_train = get_labels(data_dir_train)
labels_val = get_labels(data_dir_val)


transforms_train = v2.Compose([
    v2.Resize((244, 244)),
    v2.RandomResizedCrop(size=(224, 224), antialias=True),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomVerticalFlip(p=0.5),
    # v2.RandomRotation(degrees=(-20, 20)),
    v2.RandomAffine(degrees=(-10, 10), translate=(0.1, 0.1), scale=(0.9, 1.1)),
    v2.RandomErasing(p=0.5, scale=(0.1,0.15)),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std =[0.229, 0.224, 0.225])
])
transforms_val = v2.Compose([
    v2.Resize((244, 244)),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std =[0.229, 0.224, 0.225])
])

In [ ]:
cls_mapping = [
    'barrel_jellyfish',
    'blue_jellyfish',
    'compass_jellyfish',
    'lions_mane_jellyfish',
    'mauve_stinger_jellyfish',
    'Moon_jellyfish'
]
encoding_mapping = { c: idx for idx, c in enumerate(cls_mapping) }
decoding_mapping = { idx: c for idx, c in enumerate(cls_mapping) }

def encode(raw_cls):
    return encoding_mapping[raw_cls]

def decode(encoded_cls):
    return decoding_mapping[encoded_cls]

In [ ]:
class JellifishDS(Dataset):
    labels = []
    transforms = []
    
    def __init__(self, labels, transforms=None):
        """
        labels is a list of (raw_jellifish_class, file_path)
        """
        self.labels = labels
        self.transforms = transforms
        
    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self, idx):
        raw_cls, img_path = self.labels[idx]

        img_tensor = read_image(img_path)
        img_tensor = img_tensor.float() / 255.0

        if self.transforms:
            img_tensor = self.transforms(img_tensor)

        encoded_cls = encode(raw_cls)
        
        return img_tensor, encoded_cls

In [ ]:
ds_train = JellifishDS(labels_train, transforms=transforms_train)
ds_valid = JellifishDS(labels_val, transforms=transforms_val)

dl_train = DataLoader(ds_train, batch_size=16, shuffle=True)
dl_valid = DataLoader(ds_valid, batch_size=16)


dataloaders = {
    'train': dl_train,
    'val': dl_valid
}

print('input shape dataset:', ds_train[0][0].shape)
i = next(iter(dl_train))
print('input shape dataloader train batch:', i[0].shape)

In [ ]:
from torchvision import datasets

transforms = v2.Compose([
    v2.PILToTensor(),
    v2.Resize((244,244))
])

ds_train_1 = datasets.ImageFolder(root=data_dir_train, transform=transforms)
ds_valid_1 = datasets.ImageFolder(root=data_dir_val, transform=transforms)
ds_train_1[0][0].shape

## 2. Loss function und Model Definieren
- Wie ermitteln wir wie gut unsere Vorhersage ist? Wie vermitteln wir unserem Model wie es sich verbessern soll? 
- Wir machen Transfer learning. Dafür brauchen wir ein vortainiertes Resnet oder ähnliches Model.
    - Das Grundlegende Model müssen wir anpassen um einen Output mit passender 'shape' zu generieren. Was bedeutet das genau?
- Printe das Model mit torchsummary `print(summary(model, input_size=(3,244,244), device='cpu'))`. Was sieht man?
### Hinweise
- Um vortrainierte Modelle einzubinden gibt es die torchvision (`torchvision.models`) oder timm library
    - z.B. `from torchvision.models import efficientnet_v2_s, EfficientNet` oder `import timm => timm.create_model(...)`
    - bei timm kann man sich mit `timm.list_models()` eine übersicht loggen 
- Welchen Input braucht die loss function?
      - der output des Models muss zu einem der Inputs der loss function passen. Der andere Input der loss function kommt durch den Dataset

In [ ]:
!pip3 install timm

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
from torchsummary import summary
from torchvision.models import efficientnet_v2_s, EfficientNet, EfficientNet_V2_S_Weights, efficientnet_b0, EfficientNet_B0_Weights

class JellyNet(nn.Module):
    def __init__(self, n_classes):
        super().__init__()

        #self.basemodel = efficientnet_v2_s(weights=EfficientNet_V2_S_Weights)
        self.basemodel = efficientnet_b0(weights=EfficientNet_B0_Weights)
        
        classifier = self.basemodel.classifier
        #print(classifier)
        
        in_features = [m for m in classifier.modules()]
        in_features = in_features[-1].in_features
        
        new_classifier = nn.Sequential(
            nn.Dropout(p=0.2, inplace=True),
            nn.Linear(in_features, 128),
            nn.Linear(128, n_classes)
        )
        #print(new_classifier)
        self.basemodel.classifier = new_classifier

    def forward(self, batch):
        return self.basemodel(batch)

model = JellyNet(6)
print(summary(model, input_size=(3,244,244), device='cpu'))

## Trail (+ Val Loop)
- In purem PyTorch gibt es keine höhere Abstraktion für das trainieren/validieren. Daher müssen wir dafür etwas eigenes schreiben.
- PyTorch kann nur dinge verrechnen die auf dem gleichen 'device' sind. Falls ein GPU verfügbar ist sollten wir diesen nutzen.

#### Hinweise
- ob ein GPU verfügbar ist findet man über `torch.cuda.is_available()` heraus.
- Valide device ids sind die strings `cuda` und `cpu`.
- `tensor.to(device)` schiebt den Tensor auf den device. `model.to(device)` schiebt die Gewichte des Models auf den device. Achtung, die tensor operation gibt eine neue referenz zurück. Sie ist nicht "inplace"

In [ ]:
def train_no_metrics(model, dataloaders, n_epochs, criterion, optimizer):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    loss_train = []
    loss_val = []
    
    for epoch in range(n_epochs):
        dl = dataloaders['train']
        avg_loss = 0.0
        acc = 0.0
        
        model.train()
        for i, data in tqdm(enumerate(dl), total=len(dl)):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            output = model(inputs)

            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            avg_loss += loss
        avg_loss /= (i+1)
        loss_train.append(avg_loss)

        print(f'{epoch:3g}-trai:\tavg_loss: {avg_loss:.3f}')
        avg_loss = 0.0
        
        dl = dataloaders['val']
        model.eval()
        with torch.no_grad():
            for i, val_data in tqdm(enumerate(dl), total=len(dl)):
                val_inputs, val_labels = val_data
                val_inputs = val_inputs.to(device)
                val_labels = val_labels.to(device)

                val_outputs = model(val_inputs)
                avg_loss +=  criterion(val_outputs, val_labels)
            avg_loss /= (i+1)
            print(f'{epoch:3g}-val:\tavg_loss: {avg_loss:.3f}')
            loss_val.append(avg_loss)

    return {
        'loss_train': loss_train,
        'loss_val': loss_val
    }

## 3. Optimizer + Hyperparameter definieren
- Was macht ein Optimizer?
- Welche Optimizer gibt es? Welchen nehmen wir?
- Welche Hyperparameter gibt es noch?

## 4. Putting it all together

In [ ]:
n_epochs = 20
lr = 0.001

optimizer = Adam(params=model.parameters(), lr=lr)

res = train_no_metrics(
    model=model,
    dataloaders=dataloaders,
    n_epochs=n_epochs,
    criterion=criterion,
    optimizer=optimizer
)
res

## Hinzufügen von metrics
- Loss bestimmt die Mathematische Qualität. Klassifizierungsmetriken die Praktische.
    - Uns reicht erstmal Multiclass accuracy
- Beobachte was passiert
- Was können wir alles ändern / anpassen. Mit welcher Begründung?

### Hinweise
- Für Metriken gibt es verschiedene Libraries. Wir könnten mit https://pytorch.org/torcheval/main/generated/torcheval.metrics.MulticlassAccuracy.html#torcheval.metrics.MulticlassAccuracy starten

In [ ]:
!pip3 install torcheval

In [ ]:
from torcheval.metrics import MulticlassAccuracy

def train(model, dataloaders, n_epochs, criterion, optimizer):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    loss_train = []
    loss_val = []
    acc_train = []
    acc_val = []
    metric = MulticlassAccuracy()
    
    for epoch in range(n_epochs):
        dl = dataloaders['train']
        avg_loss = 0.0
        acc = 0.0
        
        model.train()
        for i, data in tqdm(enumerate(dl), total=len(dl)):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            output = model(inputs)

            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            metric.update(output, labels)
            avg_loss += loss
        avg_loss /= (i+1)
        loss_train.append(avg_loss)
        acc = metric.compute()
        metric.reset()
        acc_train.append(acc)

        print(f'{epoch:3g}-trai:\tavg_loss: {avg_loss:.3f} acc: {acc:.3f}')
        avg_loss = 0.0
        
        dl = dataloaders['val']
        model.eval()
        with torch.no_grad():
            for i, val_data in tqdm(enumerate(dl), total=len(dl)):
                val_inputs, val_labels = val_data
                val_inputs = val_inputs.to(device)
                val_labels = val_labels.to(device)

                val_outputs = model(val_inputs)
                avg_loss +=  criterion(val_outputs, val_labels)
                metric.update(val_outputs, val_labels)
            avg_loss /= (i+1)
            loss_val.append(avg_loss)
            acc = metric.compute()
            acc_val.append(acc)
            metric.reset()
    
            print(f'{epoch:3g}-val:\tavg_loss: {avg_loss:.3f} acc: {acc:.3f}')

    return {
        'loss_train': loss_train,
        'loss_val': loss_val,
        'acc_train': acc_train,
        'acc_val': acc_val,
    }

In [ ]:
import timm

n_epochs = 20
lr = 0.0001
model = JellyNet(6)
#model = timm.create_model('resnet18', pretrained=True, num_classes=6)
optimizer = Adam(params=model.parameters(), lr=lr)

res = train(
    model=model,
    dataloaders=dataloaders,
    n_epochs=n_epochs,
    criterion=criterion,
    optimizer=optimizer
)
res